In [ ]:
!wget http://srtm.csi.cgiar.org/wp-content/uploads/files/srtm_5x5/TIFF/srtm_64_05.zip

In [ ]:
!wget http://srtm.csi.cgiar.org/wp-content/uploads/files/srtm_5x5/TIFF/srtm_64_06.zip

In [ ]:
!wget http://srtm.csi.cgiar.org/wp-content/uploads/files/srtm_5x5/TIFF/srtm_65_05.zip

In [ ]:
%load_ext lab_black

In [ ]:
from rasterio.merge import merge

directory = Path("../data/")

list_ = [
    "srtm_64_05/srtm_64_05.tif",
    "srtm_64_06/srtm_64_06.tif",
    "srtm_65_05/srtm_65_05.tif",
]

sources = list(rasterio.open(directory / filename) for filename in list_)
dest_array, out_transform = merge(sources)

In [ ]:
out_meta = sources[0].meta
out_meta.update(
    {
        "driver": "GTiff",
        "height": dest_array.shape[1],
        "width": dest_array.shape[2],
        "transform": out_transform,
    }
)

with rasterio.open("merged.tif", "w", **out_meta) as dest:
    dest.write(dest_array)

In [ ]:
merged = rasterio.open("merged.tif")

In [ ]:
show(merged)

In [ ]:
from shapely.geometry import box

shape = box(137.5, 34.5, 140.5, 37)

out_image, out_transform = rasterio.mask.mask(merged, [shape], crop=True)

out_meta = merged.meta
out_meta.update(
    {
        "driver": "GTiff",
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform,
    }
)

with rasterio.open("cropped.tif", "w", **out_meta) as dest:
    dest.write(out_image)

In [ ]:
cropped = rasterio.open("cropped.tif")

In [ ]:
show(cropped)

In [ ]:
import rasterio
from rasterio.plot import show

elevation = rasterio.open("cropped.tif")

In [ ]:
import matplotlib.pyplot as plt

from cartes.crs import PlateCarree, EPSG_6674
from cartes.utils.features import countries
from matplotlib.colors import LinearSegmentedColormap

cmap = LinearSegmentedColormap.from_list("mycmap", ["#fffcfb", "#70615a"])


class Custom(EPSG_6674):
    bbox = {
        "east_longitude": 151,
        "north_latitude": 47,
        "south_latitude": 25,
        "west_longitude": 124,
    }


fig, ax = plt.subplots(
    figsize=(10, 10),
    # dpi=300,
    subplot_kw=dict(projection=Custom()),
)
# show(elevation, ax=ax, cmap=cmap)
ax.add_feature(countries(scale="50m"))
ax.set_extent((137.5, 140.5, 34.5, 37))
ax.spines["geo"].set_visible(False)

In [ ]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

dst_crs = "EPSG:6674"

with rasterio.open("merged.tif") as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds
    )
    kwargs = src.meta.copy()
    kwargs.update(
        {"crs": dst_crs, "transform": transform, "width": width, "height": height}
    )

    with rasterio.open("transformed.tif", "w", **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.cubic_spline,
            )

In [ ]:
transformed = rasterio.open("transformed.tif")

In [ ]:
from shapely.geometry import box

shape = box(150000, -160000, 400000, 100000)

out_image, out_transform = rasterio.mask.mask(transformed, [shape], crop=True)

out_meta = merged.meta
out_meta.update(
    {
        "driver": "GTiff",
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform,
    }
)

with rasterio.open("cropped.tif", "w", **out_meta) as dest:
    dest.write(out_image)

In [ ]:
f = rasterio.open("cropped.tif")
show(f)

In [ ]:
elevation = f.read(1)

In [ ]:
import earthpy.plot as ep
import earthpy.spatial as es

hillshade = es.hillshade(elevation, azimuth=150, altitude=50)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(
    figsize=(10, 10), subplot_kw=dict(projection=Custom())
)
ep.plot_bands(hillshade, ax=ax, cbar=False)
ax.spines["geo"].set_visible(False)


In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection=Custom()))
ep.plot_bands(hillshade, ax=ax, cbar=False)
ax.spines["geo"].set_visible(False)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10), dpi=300)

ep.plot_bands(elevation, ax=ax, cmap="terrain", cbar=False, vmin=0)
ep.plot_bands(hillshade, ax=ax, alpha=0.7, cbar=False)

for sp in ax.spines.values():
    sp.set_visible(False)

fig.tight_layout()
fig.savefig("../contributions/challenge_day21.png")